In [26]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

ModuleNotFoundError: No module named 'imblearn'

In [3]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [4]:
beer_db = pd.read_sql_query('''SELECT * FROM beer_data;''', con=engine)
beer_db.head()

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(beer_db[['ibu','abv']], beer_db[['style']], random_state=48)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train

,style
1103,Bock
80,Gose
414,American IPA
844,American Stout
612,American Strong Ale
...,...
966,American Pale Ale (APA)
944,Hefeweizen
347,American IPA
1361,American IPA


In [46]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
fvals = le.fit_transform(np.ravel(beer_db[['style']]))
fvals = np.unique(fvals)
print(fvals)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


In [48]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000,
                criterion='gini', max_depth=None,
                min_samples_split=2, min_samples_leaf=1,
                min_weight_fraction_leaf=0.0, max_features='auto',
                max_leaf_nodes=None, min_impurity_decrease=0.0,
                min_impurity_split=None, bootstrap=True,
                oob_score=False, n_jobs=None, random_state=48,
                verbose=0, warm_start=False, class_weight=None,
                ccp_alpha=0.0, max_samples=None)

rfc.fit(X_train, le.transform(np.ravel(y_train)))
print(rfc)

RandomForestClassifier(n_estimators=1000, random_state=48)


In [50]:
preds = rfc.predict(X_test)
print(preds)
balanced_acc_score = balanced_accuracy_score(y_test, preds)
print(f'balanced_accuracy_score is: {balanced_acc_score}')

[80 15 19 80 55 12 13 15 21 15 13 15 36 32  8 32 13 10 36 13 10 13 55  7
 10 55 40 13 13 13  3 17 13 59 59 13 36  8  7 13 18 15 17 13 55 17  0  9
 52 50 56 43 13 13 15 13 13 89 89  8 15 13  3 63 10  6  7 18 17 13 35 13
 13  7 10 17  5 15 60 13 13 13 15 69 55  1 15  7 56 86 13 55 31 62 22 13
 13 10  3 15  9 15 86 62 17 19 54 13 16 82 13 13 10 10 10 15 17 16  3 15
 13 69 89 48 56 60 64 15 59  4 13 13  8 55 38 15  3 16 89 13  3 80 59 82
 13 19  8  7  3 15 69  4 17 86 81  7 10 13 35 89 17 13 15 13  3 86 48 55
 19 10 13 10  3 83 15 15 82 64 15 59 13 13 10 15 13  3 59 80 15 13 78  4
  7 82 15 13 58 27 15 64  6  7  7 13 59 52 15  3 10 13 15 18  5 19 42 79
 86 13 74 13 10 85 13 15  3  3 17 15 13 13 15 64 19 83  2 19 55 55 10 82
  4 15 15 89  4 13 60 13 68 57 85 40 15 12 15 15  2  7 13 43 29 15 13 89
 13 82 15 15 13 13 15 17 13 10 32 10 52 64  7 10 13 15 15  3 54 13  3 49
  7 15  3 13  3  7 10 13 10 86 19 89 13 13 55 36  7 17 19  3 19  7 10 59
 82 62 15 45 15 35 13  8 13 85 17  7 15  7  7 13 13

ValueError: Mix of label input types (string and number)

In [23]:
# Calculated the balanced accuracy score
preds = brf.predict(X_test)
balanced_acc_score = balanced_accuracy_score(y_test, preds)
print(f'balanced_accuracy_score is: {balanced_acc_score}')